In [66]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.4 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)


In [7]:
import spacy
import dotenv
from rich.console import Console

import web_rag as wr
import web_crawler as wc


In [14]:
console = Console()
dotenv.load_dotenv()

True

In [40]:
if not spacy.util.is_package("en_core_web_sm"):
    print("Downloading en_core_web_sm model...")
    spacy.cli.download("en_core_web_sm")
    print("Model downloaded successfully!")
nlp = spacy.load("en_core_web_sm")

In [41]:
chat, embedding_model = wr.get_models("openai", "gpt-4o-mini", 0.5)

In [108]:
from datetime import date
today = date.today().strftime("%B %d, %Y")
query = f"Is Elizabeth Holmes guilty?"
optimized_search_query = wr.optimize_search_query(chat, query)
console.print(optimized_search_query)

Elizabeth Holmes trial verdict guilty

In [109]:
sources = wc.get_sources(optimized_search_query, max_pages=5)
contents = wc.get_links_contents(sources)
console.print(f"retrieved {len(contents)} contents")

retrieved 5 contents

In [110]:
from itertools import groupby
from operator import itemgetter

def group_similar_sentences(sentences, similarity_threshold=0.5):
    grouped_sentences = []
    for i, sentence in enumerate(sentences):
        if i == 0:
            grouped_sentences.append([sentence])
        else:
            prev_group = grouped_sentences[-1]
            prev_sent = nlp(' '.join([s.text for s in prev_group]))
            curr_sent = nlp(sentence.text)
            if prev_sent.similarity(curr_sent) >= similarity_threshold:
                prev_group.append(sentence)
            else:
                grouped_sentences.append([sentence])
    return grouped_sentences

# Initialize an empty list to store the semantically split content
semantic_chunks = []

# Iterate through the contents
for content in contents:
    # Extract the page content
    page_content = content.get('page_content', '')
    
    if page_content:
        # Process the text with spaCy
        doc = nlp(page_content)
        
        # Split the content into sentences
        sentences = list(doc.sents)
        
        # Group similar sentences
        grouped_sentences = group_similar_sentences(sentences)
        
        for group in grouped_sentences:
            # Create a chunk with the grouped sentences and metadata
            chunk = {
                'text': ' '.join([sent.text for sent in group]),
                'metadata': {
                    'title': content.get('title', ''),
                    'source': content.get('link', '')
                }
            }
            semantic_chunks.append(chunk)

# Print the number of semantic chunks created
console.print(f"Created {len(semantic_chunks)} semantic chunks")

# Optionally, print a sample chunk to verify the structure
if semantic_chunks:
    console.print("Sample chunk:")
    console.print(semantic_chunks[0])

/var/folders/6k/6b1pw86x1yvc1_ds42gc1ms80000gq/T/ipykernel_7568/1451878299.py:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  if prev_sent.similarity(curr_sent) >= similarity_threshold:


Created 115 semantic chunks

Sample chunk:

{
    'text': "Theranos founder Elizabeth Holmes was sentenced Friday to more than 11 years in prison for fraud after
deceiving investors about the purported efficacy of her company's blood-testing technology.",
    'metadata': {
        'title': 'Theranos founder Elizabeth Holmes sentenced to more than 11 years in prison',
        'source': 
'https://www.cnbc.com/2022/11/18/former-theranos-ceo-elizabeth-holmes-sentenced-to-more-than-11-years-in-prison.htm
l'
    }
}

In [103]:
import numpy as np
import spacy

# Load the Spacy model
nlp = spacy.load('en_core_web_sm')

def process(text):
    doc = nlp(text)
    sents = list(doc.sents)
    vecs = np.stack([sent.vector / sent.vector_norm for sent in sents])

    return sents, vecs

def cluster_text(sents, vecs, threshold):
    clusters = [[0]]
    for i in range(1, len(sents)):
        if np.dot(vecs[i], vecs[i-1]) < threshold:
            clusters.append([])
        clusters[-1].append(i)
    
    return clusters

def clean_text(text):
    # Add your text cleaning process here
    return text

# Initialize the clusters lengths list and final texts list
clusters_lens = []
final_texts = []

# Process the chunk
threshold = 0.3
sents, vecs = process(contents[0].)

# Cluster the sentences
clusters = cluster_text(sents, vecs, threshold)

for cluster in clusters:
    cluster_txt = clean_text(' '.join([sents[i].text for i in cluster]))
    cluster_len = len(cluster_txt)
    
    # Check if the cluster is too short
    if cluster_len < 60:
        continue
    
    # Check if the cluster is too long
    elif cluster_len > 3000:
        threshold = 0.6
        sents_div, vecs_div = process(cluster_txt)
        reclusters = cluster_text(sents_div, vecs_div, threshold)
        
        for subcluster in reclusters:
            div_txt = clean_text(' '.join([sents_div[i].text for i in subcluster]))
            div_len = len(div_txt)
            
            if div_len < 60 or div_len > 3000:
                continue
            
            clusters_lens.append(div_len)
            final_texts.append(div_txt)
            
    else:
        clusters_lens.append(cluster_len)
        final_texts.append(cluster_txt)

SyntaxError: invalid syntax (1591802571.py, line 33)

In [111]:
# Perform semantic search using spaCy
def semantic_search(query, chunks, nlp, similarity_threshold=0.5):
    query_doc = nlp(query)
    relevant_chunks = []
    
    for chunk in chunks:
        chunk_doc = nlp(chunk['text'])
        similarity = query_doc.similarity(chunk_doc)
        
        if similarity > similarity_threshold:
            relevant_chunks.append((chunk, similarity))
    
    # Sort the relevant chunks by similarity score in descending order
    relevant_chunks.sort(key=lambda x: x[1], reverse=True)
    
    return relevant_chunks

# Perform the semantic search
relevant_results = semantic_search(optimized_search_query, semantic_chunks, nlp)

# Print the number of relevant chunks found
console.print(f"Found {len(relevant_results)} relevant chunks")

# Print the top 5 most relevant chunks (or all if less than 5)
console.print("\nTop relevant chunks:")
for i, (chunk, similarity) in enumerate(relevant_results[:5], 1):
    console.print(f"\n{i}. Similarity: {similarity:.4f}")
    console.print(f"Text: {chunk['text']}")
    console.print(f"Source: {chunk['metadata']['source']}")
    console.print(f"Title: {chunk['metadata']['title']}")

# Optionally, you can store these results for further processing
# relevant_chunks = [chunk for chunk, _ in relevant_results]


/var/folders/6k/6b1pw86x1yvc1_ds42gc1ms80000gq/T/ipykernel_7568/342810436.py:8: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = query_doc.similarity(chunk_doc)


Found 8 relevant chunks

Top relevant chunks:

1. Similarity: 0.6973

Text: Related Content
Press Release
SAN JOSE – Elizabeth A. Holmes was sentenced today to 135 months (11 years, 3 months) in federal prison for 
defrauding investors in Theranos, Inc. of hundreds of millions of dollars, announced United States Attorney 
Stephanie M. Hinds, Federal Bureau of Investigation Special Agent in Charge Robert K. Tripp, Food and Drug 
Administration (FDA) Assistant Commissioner for Criminal Investigations Catherine A. Hermsen, and U.S. Postal 
Inspection Service (USPIS) San Francisco Division Acting Inspector in Charge Kevin Rho. The sentence was handed 
down by United States District Judge Edward J. Davila.
 “Silicon Valley has seen the rise of companies whose inventions have changed the world and, through intellectual 
prowess, hard work, and sheer determination, this region continues to innovate,” said U.S. Attorney Stephanie M. 
Hinds.

...

Title: Theranos founder Elizabeth Holmes sentenced to more than 11 years in prison

5. Similarity: 0.5769

Text: Assistant U.S. Attorneys Robert S. Leach, Jeff Schenk, John C. Bostic, and Kelly Volkar are prosecuting the 
case with the assistance of Madeline Wachs, Lakisha Holliman, Sara Slattery, Elise Etter, Susan Kreider, and Leeya 
Kekona. The prosecution is the result of an investigation by the FBI, USPIS, and the FDA Office of Criminal 
Investigations.

Source: 
https://www.justice.gov/usao-ndca/pr/elizabeth-holmes-sentenced-more-11-years-defrauding-theranos-investors-hundred
s

Title: Northern District of California | Elizabeth Holmes Sentenced To More Than 11 Years For Defrauding Theranos 
Investors Of Hundreds Of Millions | United States Department of Justice

In [112]:
# Initialize an empty string to store the formatted chunks
formatted_chunks = ""

# Iterate through the relevant chunks and format them
for chunk, similarity in relevant_results:
    formatted_chunk = f"""
    <source>
    <url>{chunk['metadata']['source']}</url>
    <title>{chunk['metadata']['title']}</title>
    <text>{chunk['text']}</text>
    </source>
    
    """
    formatted_chunks += formatted_chunk

# Print the formatted chunks string
console.print("Formatted chunks:")
console.print(formatted_chunks)

# Optionally, you can store this formatted_chunks string for further use



Formatted chunks:

<source>
    <url>https://www.justice.gov/usao-ndca/pr/elizabeth-holmes-sentenced-more-11-years-defrauding-theranos-investor
s-hundreds</url>
    <title>Northern District of California | Elizabeth Holmes Sentenced To More Than 11 Years For Defrauding 
Theranos Investors Of Hundreds Of Millions | United States Department of Justice</title>
    <text>Related Content
Press Release
SAN JOSE – Elizabeth A. Holmes was sentenced today to 135 months (11 years, 3 months) in federal prison for 
defrauding investors in Theranos, Inc. of hundreds of millions of dollars, announced United States Attorney 
Stephanie M. Hinds, Federal Bureau of Investigation Special Agent in Charge Robert K. Tripp, Food and Drug 
Administration (FDA) Assistant Commissioner for Criminal Investigations Catherine A. Hermsen, and U.S. Postal 
Inspection Service (USPIS) San Francisco Division Acting Inspector in Charge Kevin Rho. The sentence was handed 
down by United States District Judge Edward J. Davila.
 “Silicon Valley has seen the rise of companies whose inventions have changed the world and, through intellectual 
prowess, hard work, and sheer determination, this region continues to innovate,” said U.S. Attorney Stephanie M. 
Hinds.</text>
    </source>
    
    
    <source>
    <url>https://www.justice.gov/usao-ndca/pr/elizabeth-holmes-sentenced-more-11-years-defrauding-theranos-investor
s-hundreds</url>
    <title>Northern District of California | Elizabeth Holmes Sentenced To More Than 11 Years For Defrauding 
Theranos Investors Of Hundreds Of Millions | United States Department of Justice</title>
    <text>The FBI is committed to investigating corporate fraud and working with our partners to help keep our 
capital markets working effectively,” said FBI Special Agent in Charge Robert K. Tripp. “The FBI and our partners 
worked tirelessly on this multi-year case and are proud justice has been served as a result.”
 “Today’s announcement should serve as a reminder that fraud related to medical products will not be tolerated,” 
said FDA Assistant Commissioner for Criminal Investigations Catherine A. Hermsen. “The FDA will continue to work 
with our law enforcement partners to bring to justice those who place profits above public health.”
 “Postal Inspectors worked closely with our partners at the U.S. Attorney’s Office, the FDA Office of Criminal 
Investigations, and the FBI to bring this case to court,” said USPIS San Francisco Division Acting Inspector in 
Charge Kevin Rho.</text>
    </source>
    
    
    <source>
    <url>https://www.theguardian.com/us-news/2022/nov/18/elizabeth-holmes-theranos-trial-sentencing</url>
    <title>Theranos founder Elizabeth Holmes sentenced to more than 11 years for defrauding investors | Theranos | 
The Guardian</title>
    <text>The harsh ruling – which aligned with federal sentencing guidelines but was far more severe than the 18 
months of house arrest requested by Holmes – sends a strong message from the US government to Silicon Valley, said 
Anat Alon-Beck, a law professor at Case Western Reserve University in Ohio.
</text>
    </source>
    
    
    <source>
    <url>https://www.cnbc.com/2022/11/18/former-theranos-ceo-elizabeth-holmes-sentenced-to-more-than-11-years-in-pr
ison.html</url>
    <title>Theranos founder Elizabeth Holmes sentenced to more than 11 years in prison</title>
    <text>U.S. District Court Judge Edward Davila, who presided over Holmes' trial, handed down the sentence.
</text>
    </source>
    
    
    <source>
    <url>https://www.justice.gov/usao-ndca/pr/elizabeth-holmes-sentenced-more-11-years-defrauding-theranos-investor
s-hundreds</url>
    <title>Northern District of California | Elizabeth Holmes Sentenced To More Than 11 Years For Defrauding 
Theranos Investors Of Hundreds Of Millions | United States Department of Justice</title>
    <text>Assistant U.S. Attorneys Robert S. Leach, Jeff Schenk, John C. Bostic, and Kelly Volkar are prosecuting 
the case with the assistance of Madeline W

In [113]:
prompt = wr.get_rag_prompt_template().format(query=query, context=formatted_chunks)  

In [114]:
from rich.live import Live
from rich.markdown import Markdown
from rich.panel import Panel

panel = Panel("AI response")

with Live(panel, auto_refresh=True) as live:
    full_response = ""
    for chunk in chat.stream(prompt):
        full_response += chunk.content
        panel.fit(Markdown(full_response))

panel.fit(Markdown(full_response))

Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                           Is Elizabeth Holmes Guilty?                                           │
│                                                                                                                 │
│ Yes, Elizabeth Holmes was found guilty of defrauding investors in her blood-testing startup, Theranos. She was  │
│ sentenced to 135 months (11 years and 3 months) in federal prison for her actions, which involved misleading    │
│ investors about the capabilities and performance of Theranos' blood-testing technology. This sentence was       │
│ handed down by U.S. District Judge Edward J. Davila and reflects the severity of the fraud, which involved      │
│ hundreds of millions of dollars.                                                                                │
│                                                                                                                 │
│ Holmes’ conviction serves as a significant legal precedent in the realm of corporate fraud, particularly in     │
│ Silicon Valley, where the case has drawn considerable attention. The prosecution was a collaborative effort by  │
│ the FBI, the U.S. Postal Inspection Service, and the FDA, emphasizing the seriousness with which such fraud is  │
│ treated in the medical and corporate sectors.                                                                   │
│                                                                                                                 │
│                                                   References                                                    │
│                                                                                                                 │
│  1 Northern District of California | Elizabeth Holmes Sentenced To More Than 11 Years For Defrauding Theranos   │
│    Investors Of Hundreds Of Millions                                                                            │
│    United States Department of Justice. ]8;id=534523;https://www.justice.gov/usao-ndca/pr/elizabeth-holmes-sentenced-more-11-years-defrauding-theranos-investors-hundreds\Read more here]8;;\.                                                         │
│  2 Theranos founder Elizabeth Holmes sentenced to more than 11 years for defrauding investors                   │
│    The Guardian. ]8;id=993177;https://www.theguardian.com/us-news/2022/nov/18/elizabeth-holmes-theranos-trial-sentencing\Read more here]8;;\.                                                                                │
│  3 Theranos founder Elizabeth Holmes sentenced to more than 11 years in prison                                  │
│    CNBC. ]8;id=246555;https://www.cnbc.com/2022/11/18/former-theranos-ceo-elizabeth-holmes-sentenced-to-more-than-11-years-in-prison.html\Read more here]8;;\.                                                                                        │
│  4 Elizabeth Holmes has started her 11-year prison sentence. Here's what to know                                │
│    NPR. ]8;id=513361;https://www.npr.org/2023/05/30/1178728092/elizabeth-holmes-prison-sentence-theranos-fraud-silicon-valley\Read more here]8;;\.                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [59]:
full_response = chat.invoke(prompt).content
console.print(Markdown(full_response))

Different Funding Stages of a Startup                                       

Startups typically progress through several distinct funding stages, each with its own characteristics, goals, and 
challenges. Understanding these stages is crucial for entrepreneurs seeking to navigate the startup landscape      
effectively. Here’s an overview of the primary funding stages:                                                     

                                                 1. Pre-Seed Stage                                                 

The pre-seed stage is the earliest phase of funding. At this point, entrepreneurs often use their own resources to 
develop their ideas. The primary goal is to assemble a team and create a minimum viable product (MVP) to showcase  
potential to attract further investment. This stage is crucial for validating the business concept and assessing   
its viability.                                                                                                     

                                                   2. Seed Stage                                                   

Seed funding is the first official equity funding stage, where startups raise capital in exchange for equity       
ownership. This funding typically ranges from a few thousand dollars to several million. The focus during this     
stage is on product development and market research, as the startup aims to validate its business model and gather 
initial customer feedback. Approximately 29% of startups fail due to a lack of capital at this stage, highlighting 
its importance.                                                                                                    

                                                 3. Series A Stage                                                 

In the Series A stage, startups seek venture capital funding to scale their operations. By this time, they should  
have a developed product and a consistent revenue stream. The typical amount raised in a Series A round ranges from
$1 million to $15 million, with an average of around $13 million. The goal here is to optimize the product and     
expand the customer base.                                                                                          

                                                 4. Series B Stage                                                 

Series B funding is for startups that have established a substantial user base and are looking to expand further.  
This round often involves larger investments from venture capitalists who want to play a more significant role in  
the company's strategy. The funds are typically used to scale operations and increase market reach.                

                                              5. Series C and Beyond                                               

Startups that have met their goals in previous stages may seek Series C funding or further rounds (Series D, etc.) 
to continue expanding or to prepare for an Initial Public Offering (IPO). This stage allows startups to solidify   
their market position and reach additional growth milestones.                                                      

                                         6. IPO (Initial Public Offering)                                          

An IPO is the process through which a startup offers its shares to the public for the first time. This stage is    
critical for providing liquidity for investors and raising significant capital to fuel further growth. The         
valuation at this stage is crucial, as it determines the stock price and overall market perception of the startup. 

Each of these stages presents unique challenges and opportunities, and the successful navigation through them is   
essential for the growth and sustainability of a startup.                                                          


                                                    References                 